# Ford-Fulkerson Algorithm for flow problems

In [1]:
import networkx as nx

The Ford-Fulkerson algorithm implement a simple solution for the MAX-FLOW problem. In case of integer capacities the time complexity is $O(m^2k_{max})$. The implementation below uses some tricks in order to improve the code, applying to every vertex the following attributes:
* $pred[j]$ is the vertex that precedes j in the augmenting path from $s$ to $j$
* $\epsilon[j]$ is the amount of 'residual flow' which can be sent to $j$ through that augmenting path

The method takes in input the graph, the source node $s$ and the terminal node $t$.

In [54]:
def ford_fulkerson(G, s, t):
    #useful variables of the graph given in input
    node_list = list(G.nodes)
    edge_list = list(G.edges)

    #number of nodes
    n = len(node_list)
    #number of edges
    m = len(edge_list)

    #set to every arcs (i,j) the attribute x, initializing null flow
    nx.set_edge_attributes(G, 0, 'x')
    #set attributes pred and epsilon to all the vertices
    nx.set_node_attributes(G, 0, 'pred')
    nx.set_node_attributes(G, 0, 'eps')
    #phi zero, value of flow x
    phi0 = 0
    #queue of labeled yet unprocessed vertices
    Q = set()

    #main loop
    while True:
        #set pred for every vertex to 0, and the symbol (+,-) representing forward or backward arc
        nx.set_node_attributes(G, 0, 'pred')
        nx.set_node_attributes(G, 0, 'symb')
        #set attributes for starting vertex
        G.nodes[s]['pred'] = s
        G.nodes[s]['eps'] = 1023
        #start vertex in the queue
        Q = {s}

        #loop
        while Q and G.nodes[t]['pred'] == 0:
            #vertex h pop by the queue Q, usually the small one
            h = min(Q)
            Q.remove(h)
            #for each unsaturated forwards arcs
            for j in G.neighbors(h):
                #check condition x[h,j] < k[h,j]
                if G[h][j]['x'] < G[h][j]['capacity']:
                    if G.nodes[j]['pred'] == 0:
                        G.nodes[j]['eps'] = min(G.nodes[h]['eps'], G[h][j]['capacity'] - G[h][j]['x'])
                        G.nodes[j]['pred'] = h
                        G.nodes[j]['symb'] = +1
                        Q.add(j)
            #for non-empty backward arcs
            for i in G.predecessors(h):
                #check condition x[i,h] > 0
                if G[i][h]['x'] > 0:
                    if G.nodes[i]['pred'] == 0:
                        G.nodes[i]['eps'] = min(G.nodes[h]['eps'], G[i][h]['x'])
                        G.nodes[i]['pred'] = h
                        G.nodes[i]['symb'] = -1
                        Q.add(i)
        
        #print(G.nodes(data=True))

        #find an augmenting path
        if G.nodes[t]['pred'] != 0:
            delta = G.nodes[t]['eps']
            phi0 += delta
            j = t
            #backward reconstruction of the augmenting path
            while j != s:
                #print(f'Evaluating node: {j}')
                i = G.nodes[j]['pred']
                symb = G.nodes[j]['symb']
                if symb > 0:
                    G[i][j]['x'] += delta
                else:
                    G[j][i]['x'] -= delta
                #update j
                j = i
        
        #if the current flow is optimal stop
        if G.nodes[t]['pred'] == 0:
            break

    return phi0      

### Test 1

In [2]:
G = nx.DiGraph()
G.add_edge(1, 2, capacity=2)
G.add_edge(1, 5, capacity=2)
G.add_edge(1, 4, capacity=2)
G.add_edge(2, 3, capacity=2)
G.add_edge(2, 4, capacity=1)
G.add_edge(2, 6, capacity=2)
G.add_edge(3, 7, capacity=2)
G.add_edge(4, 3, capacity=1)
G.add_edge(5, 2, capacity=1)
G.add_edge(5, 6, capacity=2)
G.add_edge(6, 3, capacity=1)
G.add_edge(6, 7, capacity=4)

In [55]:
vertices = list(G.nodes)
source = vertices[0]
terminal = vertices[-1]

flow = ford_fulkerson(G, source, terminal)
print('Flow for the given graph: ', flow)

Flow for the given graph:  5


### Test 2

In [56]:
G2 = nx.DiGraph()
G2.add_edge(1, 2, capacity=6)
G2.add_edge(1, 5, capacity=10)
G2.add_edge(1, 4, capacity=5)
G2.add_edge(2, 3, capacity=5)
G2.add_edge(2, 6, capacity=12)
G2.add_edge(3, 7, capacity=4)
G2.add_edge(4, 2, capacity=5)
G2.add_edge(4, 3, capacity=7)
G2.add_edge(5, 4, capacity=12)
G2.add_edge(6, 3, capacity=3)
G2.add_edge(6, 7, capacity=15)

In [60]:
vertices = list(G2.nodes)
source = vertices[0]
terminal = vertices[-1]

flow = ford_fulkerson(G2, source, terminal)
print('Flow for the given graph: ', flow)
print('Vertices info: ', G2.edges(data=True))

Flow for the given graph:  15
Vertices info:  [(1, 2, {'capacity': 6, 'x': 6}), (1, 5, {'capacity': 10, 'x': 4}), (1, 4, {'capacity': 5, 'x': 5}), (2, 3, {'capacity': 5, 'x': 0}), (2, 6, {'capacity': 12, 'x': 11}), (5, 4, {'capacity': 12, 'x': 4}), (4, 2, {'capacity': 5, 'x': 5}), (4, 3, {'capacity': 7, 'x': 4}), (3, 7, {'capacity': 4, 'x': 4}), (6, 3, {'capacity': 3, 'x': 0}), (6, 7, {'capacity': 15, 'x': 11})]
